## Thanks to Arsalan Esmaili (UW PhD student) for this example.
### If you have questions on this, highly recommend directing them to him (arsalan@uw.edu) or to Stack Overflow/Reddit.

# Web scraping to find number of beds in hospitals
In this notebook we want to retrieve number of beds in a hospital. The final goal is to give the city name and get back all hospital names and their capacity.

In [2]:
# install necessary packages
!pip install googlesearch-python 

### pip might not work. If so, try conda in terminal


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# import necessary packages
import requests
from bs4 import BeautifulSoup
import re
from googlesearch import search

The following function retrieves URLs containing the specified hospital name. It takes the hospital name and a number as inputs and returns the specified number of URLs that include the requested hospital name.

In [4]:
def find_hospital_info(hospital_name, max_results):
    query = hospital_name + " hospital number of beds"
    count = 0 
    for url in search(query):
        if count >= max_results:
            break  # Stop after reaching max_results (URLs)
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Find all occurrences of the word "bed" in the text
                info = soup.find_all(text=lambda text: text and "bed" in text.lower())
                print(f"Info found in {url}: {info}")
        except Exception as e:
            print(f"Failed to scrape {url} due to {e}")
        count += 1  # Increment the counter

In [5]:
# Example
find_hospital_info("Seattle Children's Hospital",4)

/var/folders/84/204q7trx39741y4p1_tyrh6m0000gp/T/ipykernel_18509/3818881066.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  info = soup.find_all(text=lambda text: text and "bed" in text.lower())


Info found in https://www.ahd.com/free_profile/503300/Seattle_Children's_Hospital_&_Regional_Medical_Center/Seattle/Washington/: ['Total Staffed Beds:', 'Beds and Patient Days by Unit', 'Available Beds', '(including swing beds)']
Info found in https://en.wikipedia.org/wiki/Seattle_Children%27s: ['Beds', "The hospital was founded as the seven-bed Children's Orthopedic Hospital in 1907 by ", ' dedicated a full 40-bed hospital at the same location.', '. The expansion included a new cancer and critical care unit as well as a new emergency department with 38 exam rooms. The facility added 80 new private beds in single patient rooms. The building is expected to use 47 percent less energy and 30 percent less water than similar-sized hospitals in the region.', 'In 2017 the hospital had a total of 403 beds.', ' was absorbed by the SCRI.', '500+ beds', '400-499 beds', '300-399 beds', '250-299 beds', '<250 beds']
Info found in https://www.seattlechildrens.org/careers/nursing/unit-descriptions/: [

The following function retrieves the number of beds for a specified hospital. It searches through a specified number of URLs (defaulting to 5) and returns the first matching number it finds. Please note that the result may not be accurate or up to date.

In [7]:
def search_hospital_beds(hospital_name):
    query = f"{hospital_name} hospital number of beds"
    max_results = 5  # Limit the number of results manually
    count = 0  # Counter to control results
    for url in search(query):
        if count >= max_results:
            break
        try:
            response = requests.get(url, timeout=10)  # Set timeout for responsiveness
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text()
                matches = re.findall(r'(\d+)\s*beds', text, re.I)  # Find phrases like "100 beds"
                if matches:
                    return matches[0]  # Return the first match
        except Exception as e:
            print(f"Failed to scrape {url} due to {e}")
        count += 1

    return "Information not found."

In [8]:
# Example
beds = search_hospital_beds("Seattle Children's Hospital")
print(f"Number of beds found: {beds}") # f-string can be used to have a cleaner output

Number of beds found: 175


Similar to previous function. The following function retrieves the number of beds for a specified hospital. It searches through a specified number of URLs (defaulting to 5) and returns all matching number it finds at each URL (one URL might contain more than one number). Please note that the result may not be accurate or up to date.

In [9]:
def search_hospital_beds(hospital_name):
    query = f"{hospital_name} hospital number of beds"
    max_results = 5  # Limit the number of URLs that are being scraped
    count = 0
    results = []  # List to store all matches
    for url in search(query):
        if count >= max_results:
            break
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text()
                matches = re.findall(r'(\d+)\s*beds', text, re.I)
                results.extend(matches)  # Collect all matches
        except Exception as e:
            print(f"Failed to scrape {url} due to {e}")
        count += 1

    return results if results else "Information not found."

In [10]:
# Example
beds = search_hospital_beds("Seattle Children's hospital")
print(f"Number of beds found: {beds}")

Number of beds found: ['175', '403', '499', '399', '299', '250']


Note that the numbers 403 and 499 appeared more frequently than others, suggesting that these are more likely to be the correct values.
We now use an additional function to display the scraped URLs along with the sentences containing the number of beds. This approach provides a clearer and more accurate understanding of the actual number of beds.

In [11]:
def search_hospital_beds(hospital_name):
    query = f"{hospital_name} hospital number of beds"
    max_results = 5  # Limit the number of URLs that are being scraped
    count = 0
    results = []  # List to store all matches with context

    for url in search(query):
        if count >= max_results:
            break
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text()

                # Find sentences with "beds" and include the number
                sentences = re.findall(r'([^.]*?\b\d+\s*beds\b[^.]*\.)', text, re.I)
                results.extend([(url, sentence.strip()) for sentence in sentences])  # Store URL and sentence
        except Exception as e:
            print(f"Failed to scrape {url} due to {e}")
        count += 1

    return results if results else "Information not found."

In [12]:
# Example usage
hospital_name = "Seattle Children's hospital"
search_hospital_beds(hospital_name)


[("https://www.ahd.com/free_profile/503300/Seattle_Children's_Hospital_&_Regional_Medical_Center/Seattle/Washington/",
  '2356\n\n\n\n\n\n\n\n\n            Build color coded maps based on more detailed Patient Origin data\nMore Information |\n            Sample Report\n\n\n\n\n\nOutpatient Utilization Statistics by APC\nDefinitions\n\n\n\nAPCNumber\nAPC Description\nNumberPatientClaims\nAverageCharge\nAverageCost\n\n\n\n\n5012\nClinic Visits and Related Services\n383\n$132\n$252\n\n\n5693\nLevel 3 Drug Administration\n57\n$976\n$461\n\n\n5524\nLevel 4 Imaging without Contrast\n22\n$2,919\n$1,368\n\n\n5691\nLevel 1 Drug Administration\n68\n$546\n$263\n\n\n5694\nLevel 4 Drug Administration\n18\n$1,306\n$617\n\n\n5522\nLevel 2 Imaging without Contrast\n49\n$1,335\n$363\n\n\n5024\nLevel 4 Type A ED Visits\n12\n$3,122\n$1,391\n\n\n5523\nLevel 3 Imaging without Contrast\n15\n$1,795\n$528\n\n\n5023\nLevel 3 Type A ED Visits\n13\n$1,498\n$667\n\n\n5692\nLevel 2 Drug Administration\n23\n$747\n$